In [35]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://localhost",
    "X-Title": "Prompt Playground"
}

def ask_openrouter(prompt, model="mistralai/mistral-7b-instruct"):
    url = "https://openrouter.ai/api/v1/chat/completions"
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You're a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.8
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code != 200:
        print("❌ Error:", response.status_code, response.text)
        return "API Error"

    return response.json()["choices"][0]["message"]["content"]

In [37]:
# Safe model list
models = [
    "mistralai/mistral-7b-instruct",
    "meta-llama/llama-3-8b-instruct",
    "openchat/openchat-7b",
    "nousresearch/nous-hermes-llama2-13b"
]

In [39]:
prompt_styles = {
    "sarcastic": "Respond sarcastically: {prompt}",
    "shakespeare": "Answer in Shakespearean English: {prompt}",
    "coder": "Explain like a Python developer: {prompt}",
    "funny": "Give a funny answer: {prompt}",
    "default": "{prompt}"
}

def build_prompt(user_prompt, style):
    return prompt_styles.get(style, "{prompt}").format(prompt=user_prompt)

In [41]:
import gradio as gr

def chat_app(user_input, style):
    styled_prompt = build_prompt(user_input, style)
    return ask_openrouter(styled_prompt)

gr.Interface(
    fn=chat_app,
    inputs=[
        gr.Textbox(label="Your Prompt"),
        gr.Dropdown(label="Response Style", choices=["default", "sarcastic", "shakespeare", "coder", "funny"])
    ],
    outputs="text",
    title="Prompt Engineering Playground 💥",
    description="Test prompts with different styles using free OpenRouter LLMs"
).launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
